## Imports

In [1]:
import scipy.io, random
import numpy as np
from utils import *
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd

## Utils

In [2]:
def plot_forecast(source, prediction, nrow=5, ncol=5, from_=2800, to_=None, title='Stationary'):
    n = source.shape[0]
    idx = list(range(n))
    fig, axs = plt.subplots(nrow, ncol, figsize=(15, 6), sharex=True)
    if to_ is None:
        to_ = source.shape[1]
    i = 0
    for r in range(nrow):
        for c in range(ncol):
            axs[r,c].plot(range(from_, to_), source[idx[i], from_:to_], c='b', label='Real values')
            axs[r,c].plot(range(from_, to_), prediction[idx[i], from_:to_], c='r', label='Forecasts')
            axs[r,c].grid()
            i += 1
            
    fig.tight_layout()
    handles, labels = fig.axes[0].get_legend_handles_labels()
    fig.legend(handles, labels, ncol=2, fontsize='medium', loc='lower center', bbox_to_anchor=(0.3, -0.1, 0.5, 0.5))
    plt.setp(axs[-1, :], xlabel='Time')
    fig.suptitle(f'{title} Forecasts of the First {nrow * ncol} Locations', y = 1.05)
    plt.show()

In [3]:
def read_data_from_csv(data_path):
    df = pd.read_csv(data_path)
    X = df.iloc[:, 1:].to_numpy()
    return X

## Init

In [4]:
ids, d, _ = load_pickle('data/sample.pickle')

In [ ]:
data = np.load('data/mine/data.npy')
data_std = scale(data,0,1)

In [ ]:
#  Check data pattern
i = 5
plt.figure(figsize=(20,10))
# plt.plot(data[i, :])
plt.plot(data_std[i, :])

# Stationary

## 1. Forecasting

In [ ]:
sim = load_pickle('output/st_sim.pickle')

In [ ]:
sim[0].shape, sim[1].shape, sim[2].shape

In [ ]:
plt.figure(figsize=(20,8))
plt.plot(sim[0][10, -200:], c='b')
plt.plot(sim[1][10, -200:], c='r')

plt.show()

In [ ]:
plot_forecast(sim[0], sim[1], 4, 4, 300, 500)

## 2. Shape Function

In [ ]:
F = np.log(d+1)

In [ ]:
l = 1000
plt.figure(figsize=(8,5))
plt.scatter(d[:l], F[:l, ], c='b', label='Actual SF')
plt.scatter(d[:l], sim[2][:l, 0], c='r', label='Estimated SF')
plt.legend()
plt.grid()
plt.xlabel('Distance')
plt.title('Stationary Simulation')
plt.show()

In [ ]:
plot_shape_function(d_norm, sim[2], None, 5, 5)

## 3. Multi-Step

In [ ]:
h1 = load_pickle('output/sim_h1.pickle')
h2 = load_pickle('output/sim_h5.pickle')


In [ ]:
ts = 200
fig, axs = plt.subplots(1, 2, figsize=(18,8))
axs[0].plot(h1[0][10, -ts:], c='b')
axs[0].plot(h1[1][10, -ts:], c='r')
axs[0].title.set_text('h = 1')

axs[1].plot(h2[0][10, -ts:], c='b')
axs[1].plot(h2[1][10, -ts:], c='r')
axs[].title.set_text('h = 5')
plt.show()

# Non-Stationary

In [5]:
out = load_pickle('output/nst_sim.pickle')
out[0].shape, out[1].shape, out[2].shape

((30, 500), (30, 500), (900, 1))

## 1. Forecasting

In [ ]:
plt.figure(figsize=(20,8))
plt.plot(out[0][10, -200:], c='b')
plt.plot(out[1][10, -200:], c='r')

plt.show()

In [ ]:
plot_forecast(out[0], out[1], 4, 4, 300, 500, 'Non-Stationary')

## 2. Shape function

In [8]:
F = np.log(d+1)

In [ ]:
l = 900
plt.figure(figsize=(8,5))
plt.scatter(d[:l], F[:l, ], c='b', label='Actual SF')
plt.scatter(d[:l], out[2][:l, 0], c='r', label='Estimated SF')
plt.legend()
plt.grid()
plt.xlabel('Distance')
plt.title('Non-Stationary Simulation')
plt.show()

## 3. Multi-Step

In [ ]:
h1 = load_pickle('output/nst_sim_h1.pickle')
h2 = load_pickle('output/nst_sim_h10.pickle')


In [ ]:
ts = 500
fig, axs = plt.subplots(1, 2, figsize=(18,8))
axs[0].plot(h1[0][10, -ts:], c='b')
axs[0].plot(h1[1][10, -ts:], c='r')
axs[0].title.set_text('h = 1')

axs[1].plot(h2[0][10, -ts:], c='b')
axs[1].plot(h2[1][10, -ts:], c='r')
axs[1].title.set_text('h = 5')
plt.show()

# Baselines

In [ ]:
dataset = 'nst_sim'
horizon = 1
loc, n = 30, 500

In [ ]:
#  Scale data and horizon
# data = scale(data, 0.3, 0)

In [ ]:
x_offsets = np.sort(np.concatenate((np.arange(1-horizon, 1, 1),)))
y_offsets = np.sort(np.arange(1, 1+horizon, 1))
x_ids, y_ids = [], []
min_t = abs(min(x_offsets))
max_t = abs(n - abs(max(y_offsets)))  # Exclusive
for t in range(min_t, max_t):
    x_ids.extend(t + x_offsets)
    y_ids.extend(t + y_offsets)

In [ ]:
def aggregate(v, n=7320, h=20, loc=30):
    out = [np.zeros((1, loc)) for _ in range(n)]
    cnt = [np.ones((1, loc)) for _ in range(h)] + [np.zeros((1, loc)) for _ in range(n-h)]
    L = len(y_ids)
    for l in range(L):
        i = y_ids[l] 
        out[i] += v[l, :]
        cnt[i] += 1

    outs = np.concatenate(out)
    cnts = np.concatenate(cnt)
    return (outs/cnts).transpose()

In [ ]:
def load_outputs(path, n, h, loc):
    data = np.load(path)
    input, pred, label = data['input'], data['prediction'], data['truth']
    
    
    if 'DC-RNN' in path:
        pred, label = pred.swapaxes(0,1), label.swapaxes(0,1)
        
    predictions = aggregate(pred.reshape(-1, loc), n, h, loc)
    labels = aggregate(label.reshape(-1, loc), n, h, loc)
#     print(input.shape, pred.shape, label.shape, '//', predictions.shape, labels.shape)
    
    return predictions, labels

## Visualization

In [ ]:
def compute_metrics(name, p, l, horizon, size = None):
    if size:
        p = p[:, -size:]
        l = l[:, -size:]
    # Mean absolute error
    mae = (np.abs(p - l) / horizon).mean()

    # Mean Absolute Percentage Error
    smape = 2.0 * (np.abs(l-p)/(np.abs(l+1e-10)+np.abs(p+1e-10))).mean()

    # Root Mean Squared Error
    rmse = ((p - l)**2 / horizon).mean(-1)
    rmse = np.sqrt(rmse).mean()


    return mae, smape, rmse

In [ ]:
def get_result(i, horizon, collector):
    
    for name in names:
        p, l = load_outputs(f'{name}/data/{dataset}/output/preds{i}.npz', n, horizon, loc)
        out = compute_metrics(name, p, l, horizon)
        collector[name]['mae'].append(out[0])
        collector[name]['smape'].append(out[1])
        collector[name]['rmse'].append(out[2])
    return collector

In [ ]:
names = ['DC-RNN', 'FC-GAGA', 'GMAN', 'ConvLSTM']
collector = {}
for name in names:
    collector[name] = {"mae": [], "smape": [], "rmse": []}

In [ ]:
for i in tqdm(range(100)):
    collector = get_result(i, horizon, collector)    

In [ ]:
for name in names:
    mae = np.array(collector[name]['mae']).mean()
    smape = np.array(collector[name]['smape']).mean()
    rmse = np.array(collector[name]['rmse']).mean()
    print(f"{name} & {mae:.4f} & {smape:.4f} & {rmse:.4f} \\\\")

In [ ]:
mae, smape, rmse = 0, 0, 0
iter = range(100)
for i in tqdm(iter):
    p = np.load(f'output/{dataset}/preds{i}.npy')
    l = pd.read_csv(f'data/{dataset}/csv/s{i}.csv')
    l = l.iloc[:, 1:].to_numpy()
    a, b, c = compute_metrics('Shape function', p, l , horizon)
    mae += a
    smape += b
    rmse += c

mae /= len(iter)
smape /= len(iter)
rmse /= len(iter)
print(f"Shape function & {mae:.4f} & {smape:.4f} & {rmse:.4f} \\\\")